In [1]:
import numpy as np
from tqdm.notebook import tqdm

from scipy.ndimage import convolve

from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Div
from bokeh.layouts import row
from bokeh.palettes import Colorblind

from  ipywidgets import interact
output_notebook()

from recaps.utils import euler_forward, runge_kutta2, convert2img

Loading BokehJS ...

### Building intuition on simple systems

In general, the reaction-diffusion PDEs that we saw in the previous section can equally well describe homogeneous systems that oscillate in time (systems with Hopf bifurcation), inhomogenous steady systems (systems with Turing patterns), as well as inhomogenous nonsteady systems. So what regulates the behaviour and determines the patterns? Well, the ability to accommodate various patterns depends on the complexity of the reaction term, but the exact nature of patterns depends on system parameters. 

So, if we select the reactions and set the parameters we can simulate the system and check how it behaves. That's exactly what we did in the previous section. But can we _predict_ this behaviour _without_ simulation? Just from looking at the parameters? And, more importantly, if we're interested in a very particular behaviour, is there a way to pick the corresponding parameters without checking each and every combination until we finally hit the right one? The answer is "_sort of_": we can't determine which parameters in Gray-Scott system will lead to the formation of ["U-skates"](http://mrob.com/pub/comp/xmorphia/uskate-world.html), but we can map the regions in parameter space where certain types of patterns are technically possible. Doing this would require us to remember some bits and pieces from the Linear Algebra classes. But before we start remembering let's just play around with the simpler systems and try to build up some intuition first.

### Simple homogeneous oscillators

Let's forget about the diffusion for a moment and pretend that we have a homogeneous system. Suppose that we have a system of two enzymes: $A$ promotes the formation of $B$ and $B$ catalyzes the breakdown of $A$. To keep things simple let's assume that the rate of $B$ formation is linearly dependent on the concentration of $A$ and the rate of $A$ breakdown is linearly dependent on the concentration of $B$.

<img width="300" height="50" src="images/ab_simple_interaction.png">

Let's check how the system would behave if we vary parameters $k_{AB}$ and $k_{BA}$. We'll start with the function which reflects the mass balances of the two species.


In [2]:
# ---define the system---
# mass balance: all we know about the interactions within the system goes here
def balances(_, x, p):
    """dc_a/dt = -k_ba*c_b
       dc_b/dt =  k_ab*c_a"""     
    x_a, x_b = x[0], x[1]
   
    return np.array([ p['p_ba'] * x_b,  # dc_a/dt
                      p['p_ab'] * x_a]) # dc_b/dt

This function only calculates the rates of changes in $A$ and $B$ over a single time step. So, to find how the concentrations of $A$ and $B$ change over long periods of time, we'll need to repeat the same calculation for different time steps. This time we'll use the second order [Runge Kutta](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods) method.

><font size=2>Quick recap: [numerical solutions of ordinary differential equations](recaps/numerical_solutions_recap.ipynb)</font>

In [3]:
# ---set up solver (Runge Kutta 2)---
def runge_kutta2(dxdt, x, t_span, p):
    x = x.astype(float)
    
    # preallocate/initialize
    x_span = np.zeros((len(x), len(t_span)))
    x_span[:,0] = x
        
    for it,t in enumerate(t_span[1:]):
        dt = t_span[it+1] - t_span[it]
        k1 = dxdt(t,        x,        p)*dt
        k2 = dxdt(t+0.5*dt, x+0.5*k1, p)*dt
        x += k2
        x_span[:,it+1] = x
        
    return x_span 

#---specify some useful stuff---
# time related
t0,dt,tf = 0,0.05,50
t_span = np.arange(t0,tf+dt,dt)

# initial condition
x0 = np.array([0.5,  # c_a(t=0)
               0.5]) # c_b(t=0)

# parameters
p = {'p_ba': -0.5,
     'p_ab':  0.5}
        
# ---simulate the system---
x_num = runge_kutta2(balances, x0, t_span, p)

In [4]:
# ---plot stuff!---
def get_dynamic_plots(t_span, x_num, title, labels, legend, comment=""):
    """
    plots state development over time and state limit cycle
    INPUTS:
        t_span: 1D ndarray: array containing the time instances
        x_num: (#states, #time-steps) ndarray: output of the ode-solver
        title: str: plot title
        labels: List[str]: axis labels for dynamic plot (x axis (time), y axis)
        legend: List[str]: state names
    OUTPUS:
        figure handels for dynamic plot, phase-space plot and their plot handles
    """
    # plot x1,x2 as function of time
    y_min, y_max = min(x_num.ravel()), max(x_num.ravel()) 
    plt1 = figure(title=title + ": dynamics", 
                  x_range=[t_span[0], t_span[-1]], 
                  y_range=[y_min - 0.5*(y_max-y_min), y_max + 0.5*(y_max-y_min)],
                  plot_width=450, plot_height=250)
    plt1.xaxis.axis_label = labels[0]
    plt1.yaxis.axis_label = labels[1]
    
    colors = Colorblind[max(len(x_num), min(Colorblind.keys()))]
    r1 = [
        plt1.line(
            t_span, x_num[i,:], 
            color=colors[i], line_width=2, legend_label=legend[i]
        ) for i in range(len(x_num))
    ]
    
    # phase orbit: x2 as function of x1
    x_min, x_max, y_min, y_max = min(x_num[0,:]), max(x_num[0,:]), min(x_num[1,:]), max(x_num[1,:])
    plt2 = figure(title="phase orbit",
                  x_range=[x_min - 0.3*(x_max-x_min), x_max + 0.3*(x_max-x_min)],
                  y_range=[y_min - 0.3*(y_max-y_min), y_max + 0.3*(y_max-y_min)],
                  plot_width=250, plot_height=250)
    plt2.xaxis.axis_label = legend[0]
    plt2.yaxis.axis_label = legend[1]
    r2 = plt2.line(x_num[0,:], x_num[1,:], color=colors[0], line_width=2)    
      
    return plt1, plt2, r1, r2

title = "simple homogeneous oscillator"
labels = ["time", "concentration"]
legend = ["concentration of A", "concentration of B"]
with open('recaps/comment.txt') as f:
    comment = f.read()

plt1, plt2, r1, r2 = get_dynamic_plots(
    t_span, x_num, title, labels, legend, comment=comment
)   
div = Div(width=250, text=comment)
show(row(plt1, plt2, div), notebook_handle=True)  


# add interactive sliders to check the effect of parameters and dt
def update(p_ab=0.25, p_ba=-0.25):
    t = np.arange(t0,tf+dt,dt)
    c_a, c_b = runge_kutta2(balances, x0, t, {'p_ba': p_ba, 'p_ab': p_ab})
    
    r1[0].data_source.data = {'x': t,   'y': c_a}
    r1[1].data_source.data = {'x': t,   'y': c_b}
    r2.data_source.data  = {'x': c_a, 'y': c_b}
    push_notebook()

interact(update, p_ab=(-1,1,0.05), p_ba=(-1,1,0.05));

interactive(children=(FloatSlider(value=0.25, description='p_ab', max=1.0, min=-1.0, step=0.05), FloatSlider(v…

Whenever two system parameters have opposite signs (one enzyme promotes synthesis and the other catalyzes breakdown), the system happens to be stuck in a perpetual loop: the more $B$ $\rightarrow$ the faster breakdown of $A$ $\rightarrow$ the less $A$ $\rightarrow$ the slower synthesis of $B$ $\rightarrow$ the less $B$. And when two parameters have the same signs the system "explodes"... That's not particularly interesting... So  let's make a small correction: suppose $B$ now also decays on its own with the rate proportional to its concentration:

<img width="400" height="50" src="images/ab_interaction_with_decay.png">

In [5]:
def balances(_, x, p):
    """dc_a/dt = -k_ba*c_b
       dc_b/dt =  k_ab*c_a - d_b*c_b""" 
    c_a, c_b = x[0], x[1]
    
    return np.array([p['p_ba']*c_b,
                     p['p_ab']*c_a + p['p_bb']*c_b])

p = {'p_bb': -0.10,
     'p_ba': -0.25,
     'p_ab':  0.25}

# ---simulate---
x_num = runge_kutta2(balances, x0, t_span, p)

# ---plot stuff!---
show(row(plt1, plt2), notebook_handle=True)

# add interactive sliders to check the effect of parameters and dt
def update(p_bb=-0.10, p_ab=0.25, p_ba=-0.25):
    t = np.arange(t0,tf+dt,dt)
    c_a, c_b = runge_kutta2(balances, x0, t, {'p_bb':p_bb, 'p_ba':p_ba, 'p_ab':p_ab})
    
    r1[0].data_source.data = {'x': t,   'y': c_a}
    r1[1].data_source.data = {'x': t,   'y': c_b}
    r2.data_source.data  = {'x': c_a, 'y': c_b}
    push_notebook()

interact(update, p_bb=(-1,1,0.01), p_ab=(-1,1,0.01), p_ba=(-1,1,0.01));
    

interactive(children=(FloatSlider(value=-0.1, description='p_bb', max=1.0, min=-1.0, step=0.01), FloatSlider(v…

By introducing "self-decay" we got ourselves a damper! Now we can regulate whether or when the system reaches a stable steady state. 

One thing we can still check is the effect of "self-promotion" of $A$. By now you can probably figure out yourself, what needs to be changed in the code, to account for this process. So just go ahead - modify the code above and check how it affects system's behaviour.

### What does Linear Algebra have to do with any of this

Toggling the parameters is fun, but it's definitely not the most systematic way of analyzing the system. We can do better. Luckily, since our system is linear, we can unleash all the tools that Linear Algebra has to offer (and that's a lot)! To see what we can do, let's first rewrite the system in a more Linear-Algebra-compliant (and somewhat cleaner) form:

$$ \begin{cases} \frac{dx_{A}}{dt} = -k_{BA}x_{B} \\
                 \frac{dx_{B}}{dt} =  k_{AB}x_{A} - k_{BB}x_{B}\end{cases} \rightarrow
   \begin{cases} \frac{dx_{A}}{dt} =  0 x_{A}     - k_{BA}x_{B} \\
                 \frac{dx_{B}}{dt} =  k_{AB}x_{A} - k_{BB}x_{B}\cdot x_{B} \end{cases} \rightarrow
   \begin{bmatrix} \frac{dx_{A}}{dt} \\ \frac{dx_{B}}{dt} \end{bmatrix} = 
   \begin{bmatrix} 0 & -k_{BA} \\ k_{AB} & -k_{BB} \end{bmatrix} \cdot 
   \begin{bmatrix} x_{A} \\ x_{B} \end{bmatrix} \rightarrow 
   \dot{\textbf{x}} = \textbf{A} \textbf{x} $$
   
where $\dot{\textbf{x}} = \begin{bmatrix} \frac{dx_{A}}{dt} \\ \frac{dx_{B}}{dt} \end{bmatrix}$, 
      $\textbf{x} = \begin{bmatrix} x_{A} \\ x_{B} \end{bmatrix}$ and
      $\textbf{A} = \begin{bmatrix} 0 & -k_{BA} \\ k_{AB} & -k_{BB} \end{bmatrix} = \
                    \begin{bmatrix} p_{AA} & p_{BA} \\ p_{AB} & p_{BB} \end{bmatrix}$.
      
><font size=2>
The $\textbf{A}$ matrix (not to be confused with enzyme $A$) is called the <i>state matrix</i> because it transforms the <i>state vector</i> $\textbf{x}$. But you could've also recognized it as <a href=https://en.wikipedia.org/wiki/Jacobian_matrix_and_determinant>Jacobian</a>, as it is composed of first-order partial derivatives of system ODE's with respect to each of the states. </font>

Just like solution of linear ODE would have a form $x = const\cdot e^{\lambda t}$, solution of _system_ of linear ODEs without intput terms would have a form $\textbf{x} = \textbf{v}e^{\lambda t}$ (now both $\textbf{x}$ and $\textbf{v}$ are vectors). The main question is how do we find $\textbf{v}$ and $\lambda$... Let's plug in the generic candidate solution into the system definition:

$$ \dot{\textbf{x}} = \textbf{A} \textbf{x} \rightarrow 
   \lambda \textbf{v} e^{\lambda t} = \textbf{A} \textbf{v}e^{\lambda t} \rightarrow 
   \lambda \textbf{v} = \textbf{A} \textbf{v} $$
   
Did you recognize the last expression? It says that transforming the unknown vector $\textbf{v}$ by a matrix $\textbf{A}$ is equivalent to just scaling this vector by some constant coefficient $\lambda$. Well, isn't it a textbook definition of [eigenvectors](https://en.wikipedia.org/wiki/Eigenvalues_and_eigenvectors)! For us this means that $\textbf{x} = \textbf{v}e^{\lambda t}$ can only be a solution of $\dot{\textbf{x}} = \textbf{A} \textbf{x}$ if $\textbf{v}$ is the eigenvector of $\textbf{A}$ and $\lambda$ is the corresponding eigenvalue.      
      

In [6]:
def get_jac_balances(p):
    """return A mtx (Jacobian) for given p values"""
    return np.array([[0,         p['p_ba']],
                     [p['p_ab'], p['p_bb']]])

A = get_jac_balances(p)
eigval, eigvec = np.linalg.eig(A)

print('\neigenvalues: \n{}'.format(eigval))
print('\neigenvectors: \n{}'.format(eigvec))


eigenvalues: 
[-0.05+0.24494897j -0.05-0.24494897j]

eigenvectors: 
[[0.14142136+0.69282032j 0.14142136-0.69282032j]
 [0.70710678+0.j         0.70710678-0.j        ]]


Oh, 2x2 $\textbf{A}$ matrix can have two eigenvalues and two eigenvectors... Which ones do we choose? Well, both. Since our system is linear, the [linear combination](https://en.wikipedia.org/wiki/Linear_combination) (weighted sum) of its solutions would also be a solution. So the general solution would look something like: 

$$ \textbf{x} = \alpha \textbf{v}_{1}e^{\lambda_1 t} + \beta \textbf{v}_{2}e^{\lambda_2 t}$$ 

where $\alpha$ and $\beta$ are some coefficients, that can be found as long as we know the state $\textbf{x}_0$ of the system at $t=0$:

$$ \textbf{x}_0 = \alpha \textbf{v}_{1}e^{\lambda_1 \cdot 0} + \beta \textbf{v}_{2}e^{\lambda_2 \cdot 0} = \
   \alpha\textbf{v}_1 + \beta \textbf{v}_2 = \
   \begin{bmatrix} \textbf{v}_1 & \textbf{v}_2 \end{bmatrix} \cdot \begin{bmatrix} \alpha \\ \beta \end{bmatrix}$$

In [7]:
coef = np.linalg.solve(eigvec, x0)
print('coefficients: \n{} \n{}'.format(coef[0], coef[1]))

coefficients: 
(0.3535533905932738-0.28867513459481287j) 
(0.35355339059327384+0.28867513459481287j)


In [8]:
# confirm if these coefficients indeed solve x0 = eigvec.coef
np.allclose(np.dot(eigvec, coef), x0)

True

By the way, did you notice that the calculated eigenvalues are complex? What does it mean for the ODE solution? Remember that the solution for each state $x$ would have a general form $\text{const} \cdot e^{\lambda t}$. If lambda is complex, we can write:

<center><br>
$ x = 
  \text{const} \cdot e^{\lambda t} = 
  \text{const} \cdot exp\big($ <font color="teal">$Re(\lambda t)$</font> + <font color="indianred">$i\text{ }Im(\lambda t)$    </font> $\big) = \
  \text{const} \cdot $ <font color="teal">$e ^{Re(\lambda t)}$</font> <font color="indianred">$e^{i \text{ } Im(\lambda t)} $</font> 
  $ = \text{const} \cdot$ 
  <font color="teal">$ e^{Re(\lambda t)}$</font>
  <font color="indianred">$\big(cos(Im(\lambda t)) + i\text{ } sin(Im(\lambda t))\big)$</font></center>

><font size=2>Here we used [Euler's formula](https://en.wikipedia.org/wiki/Euler%27s_formula):</font>
><img width="250" height="400" src="images/complex_numbers.png">

It appears that the <font color="indianred">__imaginary__</font> part of the eigenvalue $\lambda$ is responsible for all the oscillations (if $Im(\lambda t)$ is 0, the whole <font color="indianred">__reddish__</font> part reduces to $1$ and all the sines and cosines disappear)! And the <font color="teal">__real__</font> part of $\lambda$ is responsible for modulating the amplitude of these oscillations: if $Re(\lambda t)$ is positive, then the amplitude of the oscillations increases with time and the system "explodes"; and if $Re(\lambda t)$ is negative, the oscillations gradually dampen, and the system converges to stable steady-state. (Can you see what would happen if $Re(\lambda t) = 0$?) 

For us this means, that just by varying the parameters and checking the eigenvalues of the resulting $\textbf{A}$-matrix, we can map system behaviour in parameter space without running any simulations!

For example, if we want our system to oscillate forever, we need to find such a combination of parameters $p_{AB}$, $p_{BA}$ and $p_{BB}$ (let's keep $p_{AA} = 0$ for now), which would ensure that the eigenvalues of $\textbf{A}$ have a non-zero imaginary part and no real part. The eigenvalues of $\textbf{A}$ can be found from 

$$ \begin{vmatrix} 0-\lambda & p_{BA} \\ p_{AB} & p_{BB}-\lambda \end{vmatrix} = 0  \rightarrow \
   (0-\lambda)(p_{BB}-\lambda) - p_{BA}p_{AB} = 0 $$
   
Which gives us

$$ \lambda = \frac{p_{BB}}{2} \pm \frac{\sqrt{p_{BB}^2 + 4p_{BA}p_{AB}}}{2}$$

Therefore for periodic never-decaying oscillations we'll need $p_{BB}=0$ and {$p_{BA} > 0$ and $p_{AB} < 0$} or  {$p_{BA} < 0$ and $p_{AB} > 0$}. Similarly, we can map other parameter combinations:


In [9]:
def get_pattern_map(get_jac, p, p1_span, p1_name, p2_span, p2_name):
    """returns bokeh figure handle for a plot,
    which maps system behaviour at each combination of p1 and p2 within provided ranges
    INPUT:
      p: dict: system paramters (needed to find the A-matrix)
      get_jac: fun: function get_jac(x, p) to calculate the A-matrix
      px_span: 1D-array or list: range of values for px
      px_name: str: key in dictionary p corresponding to px
    OUTPUT:
      bokeh figure handle
      """
    values   = [[None for _ in range(len(p2_span))] for _ in range(len(p1_span))]
    patterns = [[None for _ in range(len(p2_span))] for _ in range(len(p1_span))]

    # check the eigenvalues for each combination of p1 and p2 in given ranges
    for ip1,p1 in enumerate(p1_span):
        for ip2,p2 in enumerate(p2_span):
            
            p[p1_name] = p1
            p[p2_name] = p2            
            
            A = get_jac(p)
            eigval,_ = np.linalg.eig(A)

            # yes oscillations
            if np.imag(eigval[0]) or np.imag(eigval[1]): 
                # system oscillates periodically
                if np.abs(np.real(eigval[0])) < 1e-6 and np.abs(np.real(eigval[1])) < 1e-6:
                    values[ip1][ip2] = 0.5
                    patterns[ip1][ip2] = "periodic oscillations"
                # system oscilates, oscillations increase with time
                elif np.real(eigval[0]) > 0 or np.real(eigval[1]) > 0: 
                    values[ip1][ip2] = 0.75
                    patterns[ip1][ip2] = "unstable oscillations"                
                # system oscillates, oscillations decay with time
                else:             
                    values[ip1][ip2] = 0.25
                    patterns[ip1][ip2] = "decaying oscillations"
            # no oscillations
            else:
                # system 'explodes'
                if eigval[0] > 0 or eigval[1] > 0:
                    values[ip1][ip2] = 1
                    patterns[ip1][ip2] = "unstable"
                # system stabilizes
                else:
                    values[ip1][ip2] = 0
                    patterns[ip1][ip2] = "stable"      
                        
    # get pattern map
    data = dict(image=[values],
                pattern=[patterns])
    
    plt = figure(x_range=(p2_span[0], p2_span[-1]),
                 y_range=(p1_span[0], p1_span[-1]),
                 plot_width=250, plot_height=250,
                 tooltips=[(p1_name, "$y"), (p2_name, "$x"), ("pattern", "@pattern")],
                 title="pattern map")

    plt.image(source=data, image='image', palette="Greys256", 
              x=min(p2_span), y=min(p1_span), dw=max(p2_span)-min(p2_span), dh=max(p1_span)-min(p1_span))
    plt.xaxis.axis_label = p2_name
    plt.yaxis.axis_label = p1_name
                    
    return plt

In [10]:
# ---define parameter ranges---
p_bb_span, p_ab_span = np.linspace(-1,1,201), np.linspace(-1,1,201)

# ---map system behaviour for given parameter ranges---
plt3 = get_pattern_map(get_jac_balances, p, p_bb_span, 'p_bb', p_ab_span, 'p_ab')

# ---plot stuff!---
# show dynamic plot and pattern map
div = Div(width=210,
          text="""<br>Simulate the system (use sliders) to confirm that the patterns \
                  mapped by analyzing the eigenvlues were identified correctly.<br><br>
                  Mouse over the regions on pattern map.""")
show(row(plt1, plt3, div), notebook_handle=True)

# add interactive sliders to check the effect of parameters and dt
def update(p_bb=-0.10, p_ab=0.25):
    t = np.arange(0, tf+0.05, 0.05)
    c_a, c_b = runge_kutta2(balances, x0, t, {'p_bb':p_bb, 'p_ab':p_ab, 'p_ba': -0.25})
    
    r1[0].data_source.data = {'x': t,   'y': c_a}
    r1[1].data_source.data = {'x': t,   'y': c_b}
    push_notebook()

interact(update, p_bb=(-1,1,0.005), p_ab=(-1,1,0.005));             

interactive(children=(FloatSlider(value=-0.1, description='p_bb', max=1.0, min=-1.0, step=0.005), FloatSlider(…

### How do we extend this for nonlinear systems?

Eigenvalues told us a lot about the patterns in system behaviour... Sorry, eigenvalues told us a lot about the patterns in __linear__ system behaviour (various steps of our analysis relied on the assumption that we're dealing with the linear system). Well, that's a bit meh, since, if we're looking for "interesting" patterns we should probably look into complex nonlinear systems. Is there _any_ way we can apply what we've learned about the eigenvalues to map the behaviour of nonlinear systems?

Turns out there is a way! And it involves __linearization__ of the nonlinear system. Linearization doesn't mean that we magically replace the nonlinear system with its linear analog without any loss of information. The system can only be linearized __locally__: just like any complex curve can be approximated by a straight line _in proximity of some point_ if we zoom-in close enough, - any system of nonlinear ODEs can also be approximated by a system of linear ODEs _in proximity of some point_. Which point? A decent choice is "some point in time veeery far from now" because often we're only interested in knowing how the system would behave "in the end": will it explode or will it stabilize? Formally "in the end" can mean "at the steady state", which is the same as "when all the changes cease" or $\frac{d\textbf{x}}{dt} = 0$. 

To make it sound less abstract let's work out an example. Let's add some bells and whistles to our naive linear system and see if we can still navigate through it. In our updated system:
- $A$ promotes the synthesis of $B$ and itself ($A$ is now an autocatalyst - we have introduced positive feedback);
- two units of $A$ are required for catalysis, therefore formation rates of $A$ and $B$ are proportional to the probability of encountering two units of $A$ at the same time and place (we have added nonlinearity);
- $B$ no longer promotes the breakdown of $A$, instead, it _inhibits_ the formation of $A$ (more nonlinearity added);
- both $A$ and $B$ decay on their own (more dampers added);
- $A$ is continuously replenished at a constant rate (just why not).

So all in all in have:
<img height=100 width=600 src="images/gierer_meinhardt.png">

><font size=2> After an update our system has become classic [Gierer-Meinhardt](https://link.springer.com/article/10.1007/BF00289234) [activator-inhibitor](http://www.scholarpedia.org/article/Gierer-Meinhardt_model) system without diffusion (yet)</font>

What would be the concentrations of $A$ and $B$ at the steady-state ($x_A^*$ and $x_B^*$)? 

$$ \begin{cases} \frac{dx_A}{dt} = 0 \\ \frac{dx_B}{dt} = 0 \end{cases} \rightarrow 
   \begin{cases} \rho \frac{x_A^{*2}}{x_B^*} - d_Ax_A^* + \rho_A = 0 \\ 
                 \rho x_A^{*2} - d_Bx_B^* = 0 \end{cases} $$

With some high-school algebra we can get $x_A^* = \frac{d_B + \rho_A}{d_A}$ and $x_B^* = \frac{\rho}{d_B}(\frac{d_B + \rho_A}{d_A})^2$. Great! Now what...

To carry out the same kind of analysis as we did for the linear system we need to rewrite our new system in the form $\dot{\textbf{x}} = \textbf{A} \textbf{x}$. Although, since this time we're dealing with a nonlinear system which we agreed to linearize around the steady-state, it would be correct to replace the actual state $\textbf{x}$ with its deviation from the steady state $\Delta{\textbf{x}} = \textbf{x} - \textbf{x}^*$. So, for a nonlinear system we would have $\Delta{\dot{\textbf{x}}} = \textbf{A} \Delta{\textbf{x}}$. 

Don't get intimidated by all that $\Delta$ stuff. For all that matters we'll only be interested in the $\textbf{A}$-matrix. Let's recall that $\textbf{A}$-matrix is a [Jacobian](https://en.wikipedia.org/wiki/Jacobian_matrix_and_determinant), so we can still find it for a nonlinear system: 

$$ \begin{bmatrix} \frac{d\Delta{x_A}}{dt} \\ \frac{d\Delta{x_B}}{dt} \end{bmatrix} = 
   \begin{bmatrix} \frac{\partial(\rho \frac{x_A^{2}}{x_B} - d_A x_A + \rho_A)}{\partial{x_A}} & 
                   \frac{\partial(\rho \frac{x_A^{2}}{x_B} - d_A x_A + \rho_A)}{\partial{x_B}} \\ 
                   \frac{\partial(\rho x_A^{2} - d_B x_B)}{\partial{x_A}} & 
                   \frac{\partial(\rho x_A^{2} - d_B x_B)}{\partial{x_B}} \end{bmatrix} \cdot 
   \begin{bmatrix} \Delta{x_A} \\ \Delta{x_B} \end{bmatrix} \rightarrow 
   \begin{bmatrix} \frac{d\Delta{x_A}}{dt} \\ \frac{d\Delta{x_B}}{dt} \end{bmatrix} = 
   \begin{bmatrix} 2\rho \frac{x_A}{x_B}-d_A & -\rho \frac{x_A^2}{x_B^2}\\
                   2\rho x_A & -d_B \end{bmatrix} \cdot 
   \begin{bmatrix} \Delta{x_A} \\ \Delta{x_B} \end{bmatrix} \rightarrow
   \Delta{\dot{\textbf{x}}} = \textbf{A} \Delta{\textbf{x}} $$ 

where $\textbf{A} = \begin{bmatrix} 2\rho \frac{x_A}{x_B}-d_A & -\rho \frac{x_A^2}{x_B^2}\\
                   2\rho x_A & -d_B \end{bmatrix}$ 
varies with time, as states $x_A$ and $x_B$ vary with time. Luckily we've already decided that we're only interested in system behaviour at the steady-state, where $\textbf{A}$-matrix becomes 
$ \textbf{A}^{*} = \begin{bmatrix} 2\rho \frac{x_A^*}{x_B^*}-d_A & -\rho \frac{x_A^{*2}}{x_B^{*2}}\\
                   2\rho x_A^* & -d_B \end{bmatrix}$.
If we plug in $x_A^* = \frac{d_B + \rho_A}{d_A}$ and $x_B^* = \frac{\rho}{d_B}(\frac{d_B + \rho_A}{d_A})^2$ and use a bit more high-school algebra, we can find 

$$ \textbf{A}^* = \begin{bmatrix} \frac{d_A(d_B - \rho_A)}{d_B + \rho_A} & - \frac{d_B^2d_A^2}{\rho(d_B + \rho_A)^2} \\
                                2\rho \frac{d_B + \rho_A}{d_A} & - d_B \end{bmatrix}$$
                                   
The rest is straightforward: we calculate the eigenvalues of $\textbf{A}^{*}$ and check system stability for various parameter combinations.

In [11]:
def gierer_meinhardt(_, x, p):
    a, b = x.ravel()
    
    return np.array([p['r']*a**2/b - p['d_a']*a + p['r_a'],
                     p['r']*a**2   - p['d_b']*b          ])

def get_steady_state_gierer_meinhardt(p):
    return np.array([(p['d_b'] + p['r_a'])/p['d_a'],  
                      p['r']/p['d_b'] * ((p['d_b'] + p['r_a'])/p['d_a'])**2])

def get_jac_gierer_meinhardt(p):
    return np.array(
        [[p['d_a']*(p['d_b']-p['r_a'])/(p['d_b']+p['r_a']),  -p['d_b']**2/p['r'] * (p['d_a']/(p['d_b']+p['r_a']))**2],
         [2*p['r']*(p['d_b']+p['r_a'])/p['d_a']           ,  -p['d_b']]])
    

# parameters: system 
p = {
    'r'  : 0.01,
    'r_a': 0.01,
    'd_a': 0.5,
    'd_b': 0.5
}

# time-related
t0, tf, dt = 0, 100, 0.02
t_span = np.arange(t0, tf+dt, dt)

# initial condition - slight deviation from the steady state (for linearization to work)
x0 = get_steady_state_gierer_meinhardt(p) + np.array([0.1, 0.1])

# run the simulation: use euler_forward or runge_kutta2 
x_num = euler_forward(gierer_meinhardt, x0, t_span, p)


# ---plot stuff!---
title = "gierer-meinhardt system"
labels = ["time", "concentration"]
legend = ["activator A", "inhibitor B"]
plt1, plt2, r1, r2 = get_dynamic_plots(t_span, x_num, title, labels, legend)


# ---map system behaviour for given parameter ranges---
# get paramter ranges
p_da_span, p_db_span = np.linspace(0.01,1.0,101), np.linspace(0.01,1,101)


# map system behaviour
plt3 = get_pattern_map(get_jac_gierer_meinhardt, p, p_da_span, 'd_a', p_db_span, 'd_b')


# ---combine all plots---
show(row(plt1, plt2, plt3), notebook_handle=True)

def update(r=p['r'], r_a=p['r_a'], d_a=0.5, d_b=0.5):
    t = np.arange(0, tf+dt, dt)
    p = {'r':r, 'r_a':r_a, 'd_a':d_a, 'd_b':d_b}
    x0 = get_steady_state_gierer_meinhardt(p) + np.array([0.1, 0.1])
    a, b = runge_kutta2(gierer_meinhardt, x0, t, p)
    
    r1[0].data_source.data = {'x': t,      'y': a}
    r1[1].data_source.data = {'x': t,      'y': b}
    r2.data_source.data    = {'x': a,      'y': b}
    push_notebook()

interact(update, r=(0.001,0.1,0.001), r_a=(0.001,0.1,0.001), d_a=(0.01,1,0.01), d_b=(0.01,1,0.01));

  0%|          | 0/5000 [00:00<?, ?it/s]

interactive(children=(FloatSlider(value=0.01, description='r', max=0.1, min=0.001, step=0.001), FloatSlider(va…

><font size=2>The pattern map only reflects the situation with fixed $\rho=0.01$ and $\rho_A=0.01$! <br>
For given values of $\rho=0.01$ and $\rho_A=0.01$ the system appears to be stable as long as $d_B > d_A$. Let's keep it in mind for now. We'll need it later.</font>

### What would happen if we add diffusion?

We already know that if we can rewrite the system of some nonlinear ODEs $\dot{\textbf{x}} = f(\textbf{x}, p)$ in the linearized form $\Delta\dot{\textbf{x}} = \textbf{A}^*\Delta\textbf{x}$, then the eigenvalues of $\textbf{A}^*$-matrix can tell us how the system will behave at some point in time around which it was linearized. 

Can we apply the same thinking to the systems with diffusion $\dot{\textbf{x}} = D\nabla^2 \textbf{x} + f(\textbf{x}, p)$? Absolutely! The idea is the same: we need to rewrite the system $\dot{\textbf{x}} = D\nabla^2 \textbf{x} + f(\textbf{x}, p)$ in the form $\Delta\tilde{\dot{\textbf{x}}} = \tilde{\textbf{A}}\Delta\tilde{\textbf{x}}$ ($\tilde{\textbf{x}}$ can be some kind of transforms of the original $\textbf{x}$) and check the eigenvalues of the resulting $\tilde{\textbf{A}}$-matrix! Let's see how we can do it for our activator-inhibitor system from the previous example:
                 
$$ \begin{cases} 
   \frac{dx_A}{dt} = D_R\nabla^2 x_A + \rho \frac{x_A^2}{x_B} - d_Ax_A + \rho_A \\ 
   \frac{dx_B}{dt} = D_F\nabla^2 x_B + \rho x_A^2 - d_Bx_B 
   \end{cases} \rightarrow \
   \begin{bmatrix}\frac{d\Delta x_A}{dt} \\ \frac{d\Delta x_B}{dt} \end{bmatrix} = \
   \begin{bmatrix} D_A & 0 \\ 0 & D_A \end{bmatrix} \cdot \
   \begin{bmatrix} \nabla^2 x_A \\ \nabla^2 x_B \end{bmatrix} + \
   \begin{bmatrix} 2\rho \frac{x_A}{x_B}-d_A & -\rho \frac{x_A^2}{x_B^2}\\
                   2\rho x_A & -d_B \end{bmatrix} \cdot 
   \begin{bmatrix} \Delta x_A \\ \Delta x_B \end{bmatrix}$$ 
   
Err... ok... So how exactly do we deal with $\begin{bmatrix} \nabla^2 x_A \\ \nabla^2 x_B \end{bmatrix}$?

The trick that we need to use here involves remembering that $x_A$ and $x_B$ are just functions and, as any other functions, they can be represented as weighted sums of sine-waves with set frequencies. That's exactly the idea behind [Fourier decomposition](https://en.wikipedia.org/wiki/Fourier_series):

$$ x = \int_{-\infty}^{\infty} \tilde{x}_{\omega}\big(cos(\omega t) + i \text{ }sin(\omega t)\big)d\omega =  \
       \int_{-\infty}^{\infty} \tilde{x}_{\omega} e^{i \text{ } \omega t}d\omega \rightarrow
       \int_{-\infty}^{\infty} \tilde{x}_{k} e^{i \text{ } k r}dk$$
       
where $k$ are the wavenumbers ($k = \frac{\omega t}{r}$ - total radians per unit of distance $r$), and $\tilde{x}_{k}$ are the weights (amplitudes) of corresponding wavenumbers. 

To approximate the original deviation from the steady-state $\Delta x$ _perfectly_ we'd need infinitely many sine-waves with frequencies ranging from $-\infty$ to $+\infty$. But in practice, to approximate $\Delta x$ _well enough_ high but finite number of frequencies will suffice. 

It might not be very clear how Fourier can help us linearize the system. If anything, it seems to be obfuscating things even more... Even though it might seem this way, we're, in fact, finally about to simplify things: the integral will help us to cancel out the derivatives in $\begin{bmatrix} \nabla^2 x_A \\ \nabla^2 x_B \end{bmatrix}$! 

If we plug $\int_{-\infty}^{\infty} \tilde{x}_{k} e^{i \text{ } k r}dk $ in our system definition, then with some determination we can get:

$$ \begin{cases} 
   \frac{d\tilde{x}_{A,k}}{dt} = -k^2D_R\tilde{x}_{A,k} + \rho \frac{\tilde{x}_{A,k}^2}{\tilde{x}_{B,k}} - d_A\tilde{x}_{A,k} + \rho_A \\ 
   \frac{d\tilde{x}_{B,k}}{dt} = -k^2D_F\tilde{x}_{B,k} + \rho \tilde{x}_{A,k}^2 - d_B\tilde{x}_{B,k}  
   \end{cases} \rightarrow $$
   
$$ \begin{bmatrix}\frac{d\Delta\tilde{x}_{A,k}}{dt} \\ \frac{d\Delta\tilde{x}_{B,k}}{dt} \end{bmatrix} = \
   \begin{bmatrix} 2\rho \frac{\tilde{x}_{A,k}}{\tilde{x}_{B,k}}-d_A -k^2D_A & \
                         -\rho \frac{\tilde{x}_{A,k}^2}{\tilde{x}_{B,k}^2}\\
                   2\rho \tilde{x}_{A,k} & -d_B -k^2D_B \end{bmatrix} \cdot 
   \begin{bmatrix} \Delta\tilde{x}_{A,k} \\ \Delta\tilde{x}_{B,k} \end{bmatrix} \rightarrow $$ 
   
$$ \Delta\tilde{\dot{\textbf{x}}} = \tilde{\textbf{A}}\Delta\tilde{\textbf{x}}$$

where $\tilde{\textbf{A}} = 
       \textbf{A}^* + \begin{bmatrix} -k^2D_A & 0 \\ 0 & -k^2D_B \end{bmatrix}$.

Fantastic! That's exactly what we need. You might presume that the eigenvalues of this new $\tilde{\textbf{A}}$-matrix might have some important role to play. Indeed, they do. Although this time the eigenvalues depend not only on the usual system parameters but also on the freshly introduced $k$ (wavenumber), whose role in pattern formation business is not entirely clear (yet). To better understand the roles of $k$ and eigenvalues in pattern formation, let's first clarify what does pattern formation actually mean.


### So what is a pattern?

What "in human language" you would call _spatial patterns_ can be formalized as _diffusion-driven instability_ or, more explicitly, _instability brought by diffusion to the otherwise stable homogenous system_. With that in mind notice how diffusion is conveniently "controlled" by parameter $k$: if $k$ is set to $0$ diffusion terms disappear, matrix $\tilde{\textbf{A}}$ becomes $\textbf{A}^*$, and each point in space becomes independent of its neighbours. So, for spatial patterns to form we need to ensure that at $k$ set to $0$ (no diffusion) system is stable ($Re(\lambda)$ of $\tilde{\textbf{A}}$ is negative) and at $k > 0$ (yes diffusion) system is unstable ($Re(\lambda)$ of $\tilde{\textbf{A}}$ is positive). Will this pattern be stationary like zebra stripes or ever-changing like spirals in Belousov-Zhabotinsky reaction? The imaginary part of the eigenvalues of $\tilde{\textbf{A}}$ can give us an indication.

<img width=500 height=300 src="images/spatial_pattern_condition.png">

Ok, we're almost done! All we have to do now is to express the general pattern-formation conditions above in terms of system paramters. It's somewhat tedious but quite straightforward, so not much to worry about here.

To make things more general let's rewrite our $\tilde{\textbf{A}}$-matrix as

$$ \tilde{\textbf{A}} = \textbf{A}^* + \begin{bmatrix} -k^2D_1 & 0 \\ 0 & -k^2D_2 \end{bmatrix} \
                      = \begin{bmatrix} a_{11} - k^2D_1 & a_{12} \\ a_{21} & a_{22} - k^2D_2 \end{bmatrix}$$

For spatial patterns to occur two things need to happen:
1. _System needs to be stable in the absence of diffusion_ ($k = 0$) $\to$ real part of the eigenvalues of $\textbf{A}^*$ should be negative.
>Eigenvalues of $\textbf{A}^*$ are:
>
>$$ \lambda = \frac{1}{2} \Big( (a_{11}+a_{22}) \pm \sqrt{(a_{11}+a_{22})^2  - 4(a_{11}a_{22} - a_{12}a_{21})} \Big) \
           = \frac{1}{2} \Big( tr(\textbf{A}^*)\pm \sqrt{tr(\textbf{A}^*)^2 - 4det(\textbf{A}^*)} \Big)$$
>
> Here $tr(\textbf{A}^*)$ is <a href="https://en.wikipedia.org/wiki/Trace_(linear_algebra)"> trace </a> of matrix $\textbf{A}^*$, and $det(\textbf{A}^*)$ is its <a href = "https://en.wikipedia.org/wiki/Determinant"> determinant </a>.
>
> To ensure stability ($Re(\lambda) < 0$) we need both:
 - $ tr(\textbf{A}^*) = a_{11}+a_{22} < 0$
 - $det(\textbf{A}^*) = a_{11}a_{22} - a_{12}a_{21} > 0$
    
2. _System needs to be unstable in the presence of diffusion_ ($k > 0$) $\to$ real part of any eigenvalue of $\tilde{\textbf{A}}$ should be positive. 
>Eigenvalues of $\tilde{\textbf{A}}$ are:
>
>$$ \lambda \
= \frac{1}{2} \Big( (a_{11}+a_{22} - k^2(D_1+D_2)) \pm \sqrt{\
                    (a_{11}+a_{22} - k^2(D_1+D_2))^2  - 4((a_{11}-k^2D_1)(a_{22}-k^2D_2) - a_{12}a_{21})} \Big) \
= \frac{1}{2} \Big( tr(\tilde{\textbf{A}})\pm \sqrt{tr(\tilde{\textbf{A}})^2 - 4det(\tilde{\textbf{A}})} \Big)$$
>
> To ensure instability ($Re(\lambda) > 0$) we need either:
 - $ tr(\tilde{\textbf{A}}) = a_{11}+a_{22} - k^2(D_1+D_2) > 0$. Oopsy! This would be impossible, since we just agreed that $a_{11}+a_{22} < 0$... 
 - $det(\tilde{\textbf{A}}) = (a_{11}-k^2D_1)(a_{22}-k^2D_2) - a_{12}a_{21} < 0$. For spatial patterns to appear, we really need to make sure that this condition is fulfilled, so let's check it in more detail. First, we can rewrite it as:
 >
 > $$ D_1D_2k^4 - (a_{11}D_2 - a_{22}D_1)k^2 + (a_{11}a_{22} - a_{12}a_{21}) < 0 $$
 > Again, with some high-school algebra we can find the range of wavenumbers $k$, which would ensure that the inequality above holds true:
 >
 ><img height="400" width="700" src="images/wavenumbers.png">
 >
 >Since we're only interested in _positive_ wavenumbers, critical value of $k$ should also be positive:
 > - $ \frac{a_{11}}{D_1} + \frac{a_{22}}{D_2} > 0 $
 >
 >Since we're only interested in _real_ wavenumbers, discriminant in $k^2$ expression should be positive:
 > - $ a_{11}D_2 + a_{22}D_1 > 2\sqrt{D_1D_2(a_{11}a_{22} - a_{12}a_{21})}$
 
So all in all we have the following conditions for diffusion-driven instability:
- $ a_{11}+a_{22} < 0 $
- $ a_{11}a_{22} - a_{12}a_{21} > 0 $
- $ \frac{a_{11}}{D_1} + \frac{a_{22}}{D_2} > 0 $
- $ a_{11}D_2 + a_{22}D_1 > 2\sqrt{D_1D_2(a_{11}a_{22} - a_{12}a_{21})}$

Nice! Let's check which combinaitons of $D_1$ and $D_2$ meet these requirements:


In [12]:
# parameters: system 
#(we already checked that with r=0.01, r_a=0.01 and d_b > d_a system reaches stable steady state in the absence of diffusion)
p = {'r'  : 0.01,
     'r_a': 0.01,
     'd_a': 0.2,
     'd_b': 0.6}

p_Da_span = np.arange(1e-6,2,0.01)
p_Db_span = np.arange(1e-6,2,0.01)

values   = [[None for _ in range(len(p_Db_span))] for _ in range(len(p_Da_span))]
patterns = [[None for _ in range(len(p_Db_span))] for _ in range(len(p_Da_span))]

A = get_jac_gierer_meinhardt(p)

for iDa,Da in enumerate(p_Da_span):
    for iDb, Db in enumerate(p_Db_span):
        
        if A[0,0] + A[1,1] < 0 and\
           A[0,0]*A[1,1] - A[0,1]*A[1,0] > 0 and\
           A[0,0]/Da + A[1,1]/Db > 0 and\
           A[0,0]*Db + A[1,1]*Da > 2*np.sqrt(Da*Db*(A[0,0]*A[1,1] - A[0,1]*A[1,0])):
            values[iDa][iDb] = 0
            patterns[iDa][iDb] = 'yes!'
        else:
            values[iDa][iDb] = 1
            patterns[iDa][iDb] = 'meh...'
            
# get pattern map
data = dict(image=[values], pattern=[patterns])
    
plt = figure(x_range=(p_Db_span[0], p_Da_span[-1]),
             y_range=(p_Da_span[0], p_Da_span[-1]),
             plot_width=250, plot_height=250,
             tooltips=[("D_A", "$y"), ("D_B", "$x"), ("pattern", "@pattern")],
             title="pattern map")

plt.image(source=data, image='image', palette="Greys256", 
          x=min(p_Db_span), y=min(p_Da_span), dw=max(p_Db_span)-min(p_Db_span), dh=max(p_Da_span)-min(p_Da_span))
plt.xaxis.axis_label = "D_B"
plt.yaxis.axis_label = "D_A"

show(plt)
        

Ok, it appears that any combination of slow-diffusing activator and fast-diffusing inhibitor should destabilize the otherwise stable homogenous system and lead to pattern formation. Let's see if it is indeed true:

In [13]:
from recaps.utils import euler_forward, runge_kutta2
def gierer_meinhardt_2d(t, x, p):
    x = x.reshape(p['size'])   # resize flat 1D array back into (height,width,#channels)-array
    dxdt = np.zeros(p['size']) # preallocate the accumulation terms    
    diffusion = np.zeros(p['size']) # preallocate the diffusion term
    reaction  = np.zeros(p['size']) # preallocate the reaction term
    
    # --- get diffusion term ---
    # this time we'll take into account not only direct horizontal and vertical neighbours,
    # but the diagonal neighbours as well; 
    # they'll be assigned slightely lower weights, since they're a bit far after all
    kernel = np.array([[.05,  .20, .05], 
                       [.20, -1  , .20], 
                       [.05,  .20, .05]])
    for k in range(p['size'][2]):   
        diffusion[:,:,k] = p['D'][k]/p['h']**2 * convolve(x[:,:,k], kernel, mode="nearest")
        
    # --- get reaction term ---
    x_a, x_b = x[:,:,0], x[:,:,1]
    reaction[:,:,0] =  p['r']*(x_a**2)/(x_b+1e-16)  - p['d_a']*x_a + p['r_a']
    reaction[:,:,1] =  p['r']* x_a**2               - p['d_b']*x_b 
        
    # --- get total accumulation term ---
    dxdt = diffusion + reaction
        
    return dxdt.ravel()

# parameters: system 
p = {
    'r_a': 0.01,
    'r'  : 0.01,
    'd_a': 0.2,
    'd_b': 0.6,
    'D'  :[0.01,1]
}# diffusion rates 

# parameters: spatial grid
resolution = [80,80]          # grid dimensions in pixels
p['size']  = (*resolution, 2) # (height, width, #states)
p['h']     = 0.1              # pixel size in physical units


# time-related
t0, tf, dt = 0, 100, 0.01
t_span = np.arange(t0, tf+dt, dt)

# initial condition
x_a0 = np.random.rand(*resolution)
x_b0 = np.random.rand(*resolution)
x0 = np.stack((x_a0, x_b0), axis=2).ravel()

# run the simulation: use euler_forward or runge_kutta2 
img_development = runge_kutta2(gierer_meinhardt_2d, x0, t_span, p);

  0%|          | 0/10000 [00:00<?, ?it/s]

In [14]:
# plot stuff!
img = np.zeros(p['size'][:2], dtype=np.uint32)
view = img.view(dtype=np.uint8).reshape((*p['size'][:2], 4))
view += np.flipud( convert2img(img_development.T[0], p['size']) )

pb = figure(
    x_range=(0,p['size'][0]), 
    y_range=(0,p['size'][1]),
    plot_width=p['size'][0]*4, 
    plot_height=p['size'][1]*4
)
r = pb.image_rgba(
    image=[img],
    x=0, y=0, 
    dw=p['size'][0], 
    dh=p['size'][1]
)
handle = show(pb, notebook_handle=True)  

def update(t=0):
    i = int(t/dt)
    #update data_source
    #(adjust the contrast to make patterns more visible)
    img = np.zeros(p['size'][:2], dtype=np.uint32)
    view = img.view(dtype=np.uint8).reshape((*p['size'][:2], 4))
    view += np.flipud(convert2img(img_development[:,i], p['size'], steepness=3, midpoint=0.8))
    r.data_source.data['image'] = [img]
    push_notebook(handle=handle)
    
interact(update, t=(t0,tf,dt));

interactive(children=(FloatSlider(value=0.0, description='t', step=0.01), Output()), _dom_classes=('widget-int…

Great success! There is indeed a pattern right where we expected it! An orderly structure crystallized from the initial mess. Whoa! This last part almost sounded like what we did should disagree with the [second law of thermodynamics](https://en.wikipedia.org/wiki/Second_law_of_thermodynamics), which kind of suggests that normally things should happen the other way around - order should eventually degrade into chaos. Irreversibly. Eggs don't get unscrambled. 

In reality there is no disagreement: thermodynamics deals with closed systems near equilibrium. Our system is neither closed nor is it near equilibrium. In fact, we saw that this perpetual instability (being far from equilibrium) was the main "culprit" behind the pattern formation. Luckily many systems around us are also quite far from equilibrium, so all the kinds of peculiar patterns, from which life itself is probably the most peculiar, are free to emerge without ever being in disagreement with thermodynamics. 

Can we try to distill the general conditions for instability leading to pattern formation? In human language this time... Sure! First, the system needs to be kept **open** to external forces. In Gierer-Meinhardt system we have explicit input term $\rho_A$ to continuously replenish the activator. In predator-prey systems we have implicit assumption that prey grows on some mysterious food source which never gets depleted. If we're talking about the Earth, you can easily recognize this mysterious food source as the Sun. The energy of the Sun is captured by the photosynthetic organisms and dissipated among the rest through the food chains.

Second, patterns are formed in systems with **positive and negative feedbacks** which amplify some chance fluctuations and dampen others. In Gierer-Meinhard and Belousov-Zhabotinsky systems we have obvious autocatalytic activators and damping inhibitors. You can speculate that self-reinforcing autocatalytic processes should be especially important for inducing instability. In living systems the growth of organisms with the rates proportional to the biomass of these organisms (basic idea behind the growth kinetics) can be considered an autocatalytic process. In [water-scarce areas](https://www.annualreviews.org/doi/full/10.1146/annurev-conmatphys-033117-053959) the flow of groundwater predominantly to the patches with high vegetation, which further boosts growth in these patches, can also be considered an autocatalytic process. Similar autocatalytic processes can be found in systems which have nothing to do with the natural world. In economics such processes pass under the name of [increasing returns](http://dimetic.dime-eu.org/dimetic_files/Lect%204%20to%20Cowan%20-%20Arthur.pdf) and are characteristic to high-tech industries that process information rather than material resources. One iconic example of positive feedback in economics is the confrontation between VHS and Beta videotape formats in the mid-1970s. Video stores didn't want to stock tapes in two formats. Regular people didn't want to have two different players. So everyone had an understandable incentive to go for whichever format seemed to be the market leader. A slight leading difference in the market share that VHS managed to score initially turned into a huge difference over time and eventually made VHS the sole videotape format. 

Of course, the example with VHS doesn't really illustrate the idea behind spatial pattern formation - at the end, one of the "reactants" went completely extinct, and the whole system became boringly homogenous. What this system was missing was **diffusion** limitation. In a hypothetical world where information diffuses with a speed of a pigeon post and decisions are mainly shaped by the loudest guy at the local market, a patch-blanket-like pattern of VHS- and Beta-dominated clusters could have been quite plausible. For some particular cases we can be even more specific. Recall how we found that the patterns in Gierer-Meinhardt system would only form if activator would diffuse much slower than an inhibitor. This requirement holds true for all activator-inhibitor systems (this was shown for the first time by, you guessed it, [Gierer and Meinhardt](https://philpapers.org/archive/GIEATO-2.pdf)), and the logic behind it is quite straightforward: slow diffusion of activator ensures that chance fluctuations are amplified locally, whereas fast diffusion of inhibitor prevents local damping of activator and prevents activator from spreading. As a result, we have short-range amplification and long-range smoothing - the ingredients for spatial pattern formation!

Now that you know the basic mathematical recipe behind pattern formation you can easily dive into some more exotic systems. One fascinating thing about knowing the underlying maths is that it frees you from being constrained by the patterns that are known to exist in the physical world. You can just go ahead and explore things! For inspiration here are some super mesmerizing patterns generated by amazing people on internets!

<table>
<td> 
        <img width="300" src="images/gray-scott-coupled-replication.gif"> 
    <body><center> <br>Two coupled Gray-Scott systems <br> (by <a href="https://www.youtube.com/watch?v=c9EoI9tw6NE">Tim Hutton</a>)</center> </body>
<td>    

<td>
    <img width="300" src="images/coupled-worms.gif">
    <body><center> <br>Two coupled "worms" systems <br>(by <a href="https://www.youtube.com/watch?v=omydq4RsiUQ">Cornus Ammonis</a>)</center> </body>
<td>

<td>
    <img width="300" src="images/gray-scott-history.gif">
    <body><center> <br>Gray-Scott with history (and more!) <br> (by  <a href="https://www.youtube.com/watch?v=7nN8aQ-AR18">Dan Wills</a>)</center> </body>
<td>   
</table>
